In [25]:
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import roc_auc_score
import numpy as np

import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

import optuna
from optuna import Trial
from optuna.samplers import TPESampler

In [26]:
IS_CUSTOM = True
USE_VALID = True
show=True
use = [
    'KnowledgeTag',
    'KnowledgeTag_answerCode_median',
    'KnowledgeTag_answerCode_sum',
    'KnowledgeTag_answerCode_var',
    'assessmentItemID_answerCode_median',
    'assessmentItemID_elo_pred',
    'dayofweek_answerCode_count',
    'dayofweek_answerCode_var',
    'elapsedTime',
    'feature_ensemble_elo_pred',
    'hour',
    'hour_answerCode_count',
    'mid3',
    'testId_elapsedTime_median',
    'testId_elo_pred',
    'userID_answerCode_median',
    'userID_answerCode_sum',
    'userID_first3_answerCode_count',
    'userID_first3_answerCode_mean',
    'userID_first3_answerCode_var',
    'user_acc',
    'user_correct_answer',
    'user_total_answer'
 ]+['answerCode']

In [27]:
train = pd.read_csv("preprocessed_train_data.csv")[use]
_test = pd.read_csv("preprocessed_test_data.csv")[use]
val = _test[_test.answerCode!=-1]
te = _test[_test.answerCode==-1]
x_train = train.drop("answerCode",axis=1)
y_train = train.answerCode
x_valid = val.drop("answerCode",axis=1)
y_valid = val.answerCode
x_test = te.drop("answerCode",axis=1)
y_test = pd.read_csv("custom_answer.csv").prediction
print(f"cols : {x_train.columns}\n\
      x_train: {x_train.shape}\t y_train: {y_train.shape}\n\
      x_valid: {x_valid.shape}\t y_valid: {y_valid.shape}\n\
      x_test : {x_test.shape}\t y_test : {y_test.shape}\n\
")

cols : Index(['KnowledgeTag', 'KnowledgeTag_answerCode_median',
       'KnowledgeTag_answerCode_sum', 'KnowledgeTag_answerCode_var',
       'assessmentItemID_answerCode_median', 'assessmentItemID_elo_pred',
       'dayofweek_answerCode_count', 'dayofweek_answerCode_var', 'elapsedTime',
       'feature_ensemble_elo_pred', 'hour', 'hour_answerCode_count', 'mid3',
       'testId_elapsedTime_median', 'testId_elo_pred',
       'userID_answerCode_median', 'userID_answerCode_sum',
       'userID_first3_answerCode_count', 'userID_first3_answerCode_mean',
       'userID_first3_answerCode_var', 'user_acc', 'user_correct_answer',
       'user_total_answer'],
      dtype='object')
      x_train: (2266586, 23)	 y_train: (2266586,)
      x_valid: (258626, 23)	 y_valid: (258626,)
      x_test : (744, 23)	 y_test : (744,)



In [28]:
"""
- 0.8532089220400976
{
'num_leaves': 487, 
'colsample_bytree': 0.9898994493235835, 
'reg_alpha': 0.704839604094931, 
'reg_lambda': 1.2243660267489118, 
'max_depth': 14, 
'learning_rate': 0.005560845384471103, 
'n_estimators': 2734, 
'min_child_samples': 95, 
'subsample': 0.6133640223539144}

- 0.853260041035406
{
'num_leaves': 537, 
'colsample_bytree': 0.927298228613509, 
'reg_alpha': 0.7385388600320132, 
'reg_lambda': 1.3878762058449499, 
'max_depth': 15, 
'learning_rate': 0.03361097991091424, 
'n_estimators': 2939, 
'min_child_samples': 94, 
'subsample': 0.7607938897056719}

- 0.8533994202223782
{'num_leaves': 958, 
'colsample_bytree': 0.9961314877617709, 
'reg_alpha': 0.7087106375506723, 
'reg_lambda': 5.433954982907128, 
'max_depth': 14, 
'learning_rate': 0.014575324851736808, 
'n_estimators': 2838, 
'min_child_samples': 92, 
'subsample': 0.93811346165043}

"""

"\n- 0.8532089220400976\n{\n'num_leaves': 487, \n'colsample_bytree': 0.9898994493235835, \n'reg_alpha': 0.704839604094931, \n'reg_lambda': 1.2243660267489118, \n'max_depth': 14, \n'learning_rate': 0.005560845384471103, \n'n_estimators': 2734, \n'min_child_samples': 95, \n'subsample': 0.6133640223539144}\n\n- 0.853260041035406\n{'num_leaves': 537, 'colsample_bytree': 0.927298228613509, 'reg_alpha': 0.7385388600320132, 'reg_lambda': 1.3878762058449499, 'max_depth': 15, 'learning_rate': 0.03361097991091424, 'n_estimators': 2939, 'min_child_samples': 94, 'subsample': 0.7607938897056719}\n\n- 0.8533994202223782\n{'num_leaves': 958, 'colsample_bytree': 0.9961314877617709, 'reg_alpha': 0.7087106375506723, 'reg_lambda': 5.433954982907128, 'max_depth': 14, 'learning_rate': 0.014575324851736808, 'n_estimators': 2838, 'min_child_samples': 92, 'subsample': 0.93811346165043}\n\n"

In [29]:
# random sampler
sampler = TPESampler(seed=10)

# define function
def objective(trial):

    lgbm_param = {
        'objective': 'regression',
        'verbose': -1,
        'metric': 'mse', 
        'num_leaves': trial.suggest_int('num_leaves', 900, 1024, step=1, log=True), 
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.9, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.6, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 5.0, 8.0),
        'max_depth': trial.suggest_int('max_depth',10, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-3, 1e-1),
        'n_estimators': trial.suggest_int('n_estimators', 2800, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 90, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.8, 1.0),
    }

    # Generate model
    model_lgbm = LGBMRegressor(**lgbm_param)
    model_lgbm = model_lgbm.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], 
                           verbose=0, early_stopping_rounds=25)
                           
    # * 평기 지표이다.
    # 원하는 평가 지표에 따라 사용하면 된다.                         
    auc = roc_auc_score(y_valid, model_lgbm.predict(x_valid))
    return auc

optuna_lgbm = optuna.create_study(direction='maximize', sampler=sampler)

# * n_trials의 경우 optuna를 몇번 실행하여 hyper parameter를 찾을 것인지를 정한다.
# 50으로 설정해도 유의미한 값이 나온다.
optuna_lgbm.optimize(objective, n_trials=10)

[I 2022-12-07 15:38:33,360] A new study created in memory with name: no-name-4158710e-72de-4b05-9849-5dfbd8cf6e8c
[I 2022-12-07 15:45:15,525] Trial 0 finished with value: 0.8526316591074485 and parameters: {'num_leaves': 994, 'colsample_bytree': 0.9020751949359401, 'reg_alpha': 0.8534592939705101, 'reg_lambda': 7.246411647615836, 'max_depth': 12, 'learning_rate': 0.002815744802026466, 'n_estimators': 2839, 'min_child_samples': 98, 'subsample': 0.8307656291045064}. Best is trial 0 with value: 0.8526316591074485.
[I 2022-12-07 15:46:06,196] Trial 1 finished with value: 0.8530886418091106 and parameters: {'num_leaves': 910, 'colsample_bytree': 0.9685359818367797, 'reg_alpha': 0.9813573384779746, 'reg_lambda': 5.011844798983743, 'max_depth': 13, 'learning_rate': 0.04219314744806924, 'n_estimators': 2923, 'min_child_samples': 97, 'subsample': 0.8538384353032954}. Best is trial 1 with value: 0.8530886418091106.
[I 2022-12-07 15:47:49,604] Trial 2 finished with value: 0.8532739575490339 and p

In [33]:
optuna_lgbm.best_params

{'num_leaves': 1014,
 'colsample_bytree': 0.9714575783397691,
 'reg_alpha': 0.8170177472045045,
 'reg_lambda': 5.426510142804581,
 'max_depth': 12,
 'learning_rate': 0.022298067722039322,
 'n_estimators': 2888,
 'min_child_samples': 94,
 'subsample': 0.9182433522874811}

In [34]:
best_param = optuna_lgbm.best_params
best_model = LGBMRegressor(**best_param)
best_model.fit(x_train, y_train, eval_set=[(x_train,y_train),(x_valid,y_valid)], verbose=1 ,early_stopping_rounds=25)
auc = roc_auc_score(y_valid, best_model.predict(x_valid))
auc

[1]	training's l2: 0.222712	valid_1's l2: 0.22192
Training until validation scores don't improve for 25 rounds
[2]	training's l2: 0.219408	valid_1's l2: 0.218697
[3]	training's l2: 0.216244	valid_1's l2: 0.215613
[4]	training's l2: 0.213241	valid_1's l2: 0.212693
[5]	training's l2: 0.210343	valid_1's l2: 0.20987
[6]	training's l2: 0.207564	valid_1's l2: 0.207165
[7]	training's l2: 0.204905	valid_1's l2: 0.204578
[8]	training's l2: 0.202356	valid_1's l2: 0.202104
[9]	training's l2: 0.199917	valid_1's l2: 0.199736
[10]	training's l2: 0.197579	valid_1's l2: 0.197477
[11]	training's l2: 0.195343	valid_1's l2: 0.195312
[12]	training's l2: 0.193195	valid_1's l2: 0.193238
[13]	training's l2: 0.191139	valid_1's l2: 0.191252
[14]	training's l2: 0.189323	valid_1's l2: 0.189506
[15]	training's l2: 0.187462	valid_1's l2: 0.187705
[16]	training's l2: 0.185656	valid_1's l2: 0.185969
[17]	training's l2: 0.183921	valid_1's l2: 0.184293
[18]	training's l2: 0.182284	valid_1's l2: 0.182715
[19]	training'

[159]	training's l2: 0.138902	valid_1's l2: 0.144552
[160]	training's l2: 0.138857	valid_1's l2: 0.144536
[161]	training's l2: 0.138816	valid_1's l2: 0.14452
[162]	training's l2: 0.138774	valid_1's l2: 0.144504
[163]	training's l2: 0.138733	valid_1's l2: 0.144488
[164]	training's l2: 0.138695	valid_1's l2: 0.144474
[165]	training's l2: 0.138654	valid_1's l2: 0.144459
[166]	training's l2: 0.138612	valid_1's l2: 0.144445
[167]	training's l2: 0.138573	valid_1's l2: 0.144436
[168]	training's l2: 0.138535	valid_1's l2: 0.144426
[169]	training's l2: 0.138492	valid_1's l2: 0.144409
[170]	training's l2: 0.138448	valid_1's l2: 0.144394
[171]	training's l2: 0.138405	valid_1's l2: 0.144381
[172]	training's l2: 0.138366	valid_1's l2: 0.144369
[173]	training's l2: 0.138324	valid_1's l2: 0.144351
[174]	training's l2: 0.138288	valid_1's l2: 0.144338
[175]	training's l2: 0.138246	valid_1's l2: 0.144324
[176]	training's l2: 0.138207	valid_1's l2: 0.144309
[177]	training's l2: 0.138174	valid_1's l2: 0.1

[316]	training's l2: 0.134384	valid_1's l2: 0.143387
[317]	training's l2: 0.134371	valid_1's l2: 0.143384
[318]	training's l2: 0.134344	valid_1's l2: 0.14338
[319]	training's l2: 0.134327	valid_1's l2: 0.143375
[320]	training's l2: 0.134309	valid_1's l2: 0.143374
[321]	training's l2: 0.134294	valid_1's l2: 0.143372
[322]	training's l2: 0.134279	valid_1's l2: 0.143367
[323]	training's l2: 0.134264	valid_1's l2: 0.143365
[324]	training's l2: 0.134253	valid_1's l2: 0.143364
[325]	training's l2: 0.134244	valid_1's l2: 0.143362
[326]	training's l2: 0.134225	valid_1's l2: 0.143357
[327]	training's l2: 0.134207	valid_1's l2: 0.143355
[328]	training's l2: 0.134185	valid_1's l2: 0.143354
[329]	training's l2: 0.134171	valid_1's l2: 0.143351
[330]	training's l2: 0.134153	valid_1's l2: 0.143346
[331]	training's l2: 0.134143	valid_1's l2: 0.143346
[332]	training's l2: 0.134133	valid_1's l2: 0.143346
[333]	training's l2: 0.134123	valid_1's l2: 0.143343
[334]	training's l2: 0.134102	valid_1's l2: 0.1

[474]	training's l2: 0.131841	valid_1's l2: 0.143087
[475]	training's l2: 0.131822	valid_1's l2: 0.143086
[476]	training's l2: 0.131814	valid_1's l2: 0.143085
[477]	training's l2: 0.131791	valid_1's l2: 0.143083
[478]	training's l2: 0.13178	valid_1's l2: 0.143083
[479]	training's l2: 0.131758	valid_1's l2: 0.143082
[480]	training's l2: 0.131734	valid_1's l2: 0.143078
[481]	training's l2: 0.131717	valid_1's l2: 0.143077
[482]	training's l2: 0.131696	valid_1's l2: 0.143077
[483]	training's l2: 0.131677	valid_1's l2: 0.143074
[484]	training's l2: 0.131672	valid_1's l2: 0.143074
[485]	training's l2: 0.131651	valid_1's l2: 0.14307
[486]	training's l2: 0.131628	valid_1's l2: 0.143069
[487]	training's l2: 0.131619	valid_1's l2: 0.143067
[488]	training's l2: 0.131605	valid_1's l2: 0.143067
[489]	training's l2: 0.131598	valid_1's l2: 0.143066
[490]	training's l2: 0.131588	valid_1's l2: 0.143066
[491]	training's l2: 0.131583	valid_1's l2: 0.143066
[492]	training's l2: 0.131562	valid_1's l2: 0.14

[632]	training's l2: 0.129734	valid_1's l2: 0.142973
[633]	training's l2: 0.129717	valid_1's l2: 0.142973
[634]	training's l2: 0.129701	valid_1's l2: 0.142972
[635]	training's l2: 0.129682	valid_1's l2: 0.142972
[636]	training's l2: 0.129668	valid_1's l2: 0.142971
[637]	training's l2: 0.129648	valid_1's l2: 0.142969
[638]	training's l2: 0.12963	valid_1's l2: 0.142967
[639]	training's l2: 0.129621	valid_1's l2: 0.142967
[640]	training's l2: 0.129607	valid_1's l2: 0.142967
[641]	training's l2: 0.129601	valid_1's l2: 0.142967
[642]	training's l2: 0.129579	valid_1's l2: 0.142965
[643]	training's l2: 0.129572	valid_1's l2: 0.142965
[644]	training's l2: 0.129552	valid_1's l2: 0.142962
[645]	training's l2: 0.129541	valid_1's l2: 0.142962
[646]	training's l2: 0.129522	valid_1's l2: 0.14296
[647]	training's l2: 0.129504	valid_1's l2: 0.14296
[648]	training's l2: 0.129493	valid_1's l2: 0.142959
[649]	training's l2: 0.129481	valid_1's l2: 0.14296
[650]	training's l2: 0.129467	valid_1's l2: 0.1429

[788]	training's l2: 0.127766	valid_1's l2: 0.1429
[789]	training's l2: 0.127754	valid_1's l2: 0.142902
[790]	training's l2: 0.127744	valid_1's l2: 0.142901
[791]	training's l2: 0.127741	valid_1's l2: 0.142901
[792]	training's l2: 0.127723	valid_1's l2: 0.142901
[793]	training's l2: 0.127716	valid_1's l2: 0.142902
[794]	training's l2: 0.127712	valid_1's l2: 0.142902
[795]	training's l2: 0.127704	valid_1's l2: 0.142902
[796]	training's l2: 0.127688	valid_1's l2: 0.142902
[797]	training's l2: 0.127678	valid_1's l2: 0.142902
[798]	training's l2: 0.127673	valid_1's l2: 0.142902
[799]	training's l2: 0.127668	valid_1's l2: 0.142902
[800]	training's l2: 0.127661	valid_1's l2: 0.142902
[801]	training's l2: 0.127647	valid_1's l2: 0.142902
[802]	training's l2: 0.127639	valid_1's l2: 0.142901
[803]	training's l2: 0.127625	valid_1's l2: 0.1429
[804]	training's l2: 0.127611	valid_1's l2: 0.142899
[805]	training's l2: 0.127604	valid_1's l2: 0.142898
[806]	training's l2: 0.127586	valid_1's l2: 0.1428

0.8532739575490339

In [41]:
submission = roc_auc_score(y_test, best_model.predict(x_test))
submission

0.8374432163156624

In [42]:
final_model = LGBMRegressor(**best_param)

x_train_ent = pd.concat([x_train,x_valid])
y_train_ent = pd.concat([y_train,y_valid])

x_valid_ent = x_test
y_valid_ent = y_test

sub_df = pd.read_csv("preprocessed_test_data.csv")[use]
sub_ent = sub_df[sub_df.answerCode==-1]
x_test_ent = sub_ent.drop("answerCode",axis=1)

x_train_ent.shape, y_train_ent.shape, x_valid_ent.shape, y_valid_ent.shape, x_test_ent.shape

((2525212, 23), (2525212,), (744, 23), (744,), (744, 23))

In [44]:
best_model.fit(x_train_ent, y_train_ent, eval_set=[(x_train_ent,y_train_ent),(x_valid_ent,y_valid_ent)], verbose=1 ,early_stopping_rounds=25)
auc = best_model.predict(x_test_ent)
auc

[1]	training's l2: 0.222634	valid_1's l2: 0.252899
Training until validation scores don't improve for 25 rounds
[2]	training's l2: 0.219338	valid_1's l2: 0.249016
[3]	training's l2: 0.21618	valid_1's l2: 0.24528
[4]	training's l2: 0.213185	valid_1's l2: 0.241715
[5]	training's l2: 0.210291	valid_1's l2: 0.238321
[6]	training's l2: 0.207522	valid_1's l2: 0.235129
[7]	training's l2: 0.204872	valid_1's l2: 0.231978
[8]	training's l2: 0.202331	valid_1's l2: 0.229064
[9]	training's l2: 0.199899	valid_1's l2: 0.226185
[10]	training's l2: 0.197572	valid_1's l2: 0.223537
[11]	training's l2: 0.195338	valid_1's l2: 0.220853
[12]	training's l2: 0.193196	valid_1's l2: 0.218382
[13]	training's l2: 0.19115	valid_1's l2: 0.215979
[14]	training's l2: 0.189339	valid_1's l2: 0.213964
[15]	training's l2: 0.187484	valid_1's l2: 0.211811
[16]	training's l2: 0.185687	valid_1's l2: 0.209575
[17]	training's l2: 0.18395	valid_1's l2: 0.207597
[18]	training's l2: 0.182318	valid_1's l2: 0.205687
[19]	training's 

[158]	training's l2: 0.139366	valid_1's l2: 0.162792
[159]	training's l2: 0.139326	valid_1's l2: 0.162765
[160]	training's l2: 0.139284	valid_1's l2: 0.162783
[161]	training's l2: 0.139243	valid_1's l2: 0.162746
[162]	training's l2: 0.1392	valid_1's l2: 0.162736
[163]	training's l2: 0.139159	valid_1's l2: 0.162712
[164]	training's l2: 0.139121	valid_1's l2: 0.162718
[165]	training's l2: 0.139082	valid_1's l2: 0.162712
[166]	training's l2: 0.139045	valid_1's l2: 0.16269
[167]	training's l2: 0.139	valid_1's l2: 0.162697
[168]	training's l2: 0.138964	valid_1's l2: 0.162736
[169]	training's l2: 0.13892	valid_1's l2: 0.162744
[170]	training's l2: 0.138879	valid_1's l2: 0.162762
[171]	training's l2: 0.138837	valid_1's l2: 0.162758
[172]	training's l2: 0.138802	valid_1's l2: 0.162762
[173]	training's l2: 0.138767	valid_1's l2: 0.162767
[174]	training's l2: 0.138729	valid_1's l2: 0.162785
[175]	training's l2: 0.138692	valid_1's l2: 0.162779
[176]	training's l2: 0.138658	valid_1's l2: 0.162777


array([0.62735857, 0.60385056, 0.18999974, 0.78369982, 0.28581388,
       0.87927609, 0.19252668, 0.15405085, 0.12980382, 0.89628844,
       0.62191888, 0.44740958, 0.96623993, 0.29352327, 0.69688266,
       0.95678116, 0.17181349, 0.87981371, 0.93509469, 0.18341823,
       0.94247057, 0.66588962, 0.58347143, 0.27969492, 0.25410079,
       0.80617636, 0.95747384, 0.92098915, 0.67394666, 0.77576586,
       0.87382846, 0.8395869 , 0.87511067, 0.22276847, 0.91516685,
       0.81246466, 0.23712605, 0.49690035, 0.56883623, 0.24718134,
       0.74294919, 0.17019749, 0.21813503, 0.34711494, 0.77183209,
       0.85799343, 0.61172956, 0.25957195, 0.95963776, 0.85048931,
       0.67038291, 0.3608245 , 0.50977238, 0.17734685, 0.37171804,
       0.76137577, 0.469201  , 0.97017344, 0.10663443, 0.13073516,
       0.8730528 , 0.9115341 , 0.76163171, 0.19009058, 0.15000652,
       0.52660134, 0.78704486, 0.32740224, 0.41017923, 0.64677114,
       0.76534939, 0.86989817, 0.10330228, 0.29722604, 0.25771

In [45]:
auc

array([0.62735857, 0.60385056, 0.18999974, 0.78369982, 0.28581388,
       0.87927609, 0.19252668, 0.15405085, 0.12980382, 0.89628844,
       0.62191888, 0.44740958, 0.96623993, 0.29352327, 0.69688266,
       0.95678116, 0.17181349, 0.87981371, 0.93509469, 0.18341823,
       0.94247057, 0.66588962, 0.58347143, 0.27969492, 0.25410079,
       0.80617636, 0.95747384, 0.92098915, 0.67394666, 0.77576586,
       0.87382846, 0.8395869 , 0.87511067, 0.22276847, 0.91516685,
       0.81246466, 0.23712605, 0.49690035, 0.56883623, 0.24718134,
       0.74294919, 0.17019749, 0.21813503, 0.34711494, 0.77183209,
       0.85799343, 0.61172956, 0.25957195, 0.95963776, 0.85048931,
       0.67038291, 0.3608245 , 0.50977238, 0.17734685, 0.37171804,
       0.76137577, 0.469201  , 0.97017344, 0.10663443, 0.13073516,
       0.8730528 , 0.9115341 , 0.76163171, 0.19009058, 0.15000652,
       0.52660134, 0.78704486, 0.32740224, 0.41017923, 0.64677114,
       0.76534939, 0.86989817, 0.10330228, 0.29722604, 0.25771

In [46]:
final=pd.read_csv("sample_submission.csv")
final

,id,prediction
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
739,739,0.5
740,740,0.5
741,741,0.5
742,742,0.5


In [47]:
final.prediction = auc
final

,id,prediction
0,0,0.627359
1,1,0.603851
2,2,0.190000
3,3,0.783700
4,4,0.285814
...,...,...
739,739,0.056843
740,740,0.754979
741,741,0.851335
742,742,0.805155


In [48]:
final.to_csv("LightGBM_1207.csv",index=False)

In [49]:
cat = pd.read_csv("CatBoostRegressor_1207.csv")
lgb = pd.read_csv("LightGBM_1207.csv")

In [54]:
diff = (cat.prediction-lgb.prediction).reset_index()

In [65]:
diff[diff.prediction.abs()>0.2]

,index,prediction
166,166,-0.262942


In [1]:
sota_features = {'num_leaves': 958, 
'colsample_bytree': 0.9961314877617709, 
'reg_alpha': 0.7087106375506723, 
'reg_lambda': 5.433954982907128, 
'max_depth': 14, 
'learning_rate': 0.014575324851736808, 
'n_estimators': 2838, 
'min_child_samples': 92, 
'subsample': 0.93811346165043}

sota_model = LGBMRegressor(**sota_features)
sota_model.fit(x_train, y_train, eval_set=[(x_train,y_train),(x_valid,y_valid)], verbose=1 ,early_stopping_rounds=25)
auc = roc_auc_score(y_valid, best_model.predict(x_valid))
auc

NameError: name 'LGBMRegressor' is not defined